In [1]:
import cvxopt as cvx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.spatial.distance as dt


In [2]:

# generating the data
images = np.genfromtxt("hw06_images.csv", delimiter = ",")
labels = np.genfromtxt("hw06_labels.csv", delimiter = ",")

# set the first 1000 data as training data
X_train = images[0:1000,:]
y_train = labels[0:1000].astype(int)

# set the remaining 4000 data as testing data
X_test = images[1000:5000,:]
y_test = labels[1000:5000].astype(int)

# get number of samples and number of features
N_train = len(y_train)
N_test = len(y_test)
D_train = X_train.shape[1]

In [3]:
# define Gaussian kernel function
def gaussian_kernel(X1, X2, s):
    D = dt.cdist(X1, X2)
    K = np.exp(-D**2 / (2 * s**2))
    return(K)

In [ ]:
for i in range(1, 6):
    y_train = labels[0:1000].astype(int)
    y_train[y_train != i] = -1
    y_train[y_train == i] = 1

    s = 10
    K_train = gaussian_kernel(X_train, X_train, s)
    yyK = np.matmul(y_train[:, None], y_train[None, :]) * K_train

    # set learning parameters
    C = 10
    epsilon = 1e-3

    P = cvx.matrix(yyK)
    q = cvx.matrix(-np.ones((N_train, 1)))
    G = cvx.matrix(np.vstack((-np.eye(N_train), np.eye(N_train))))
    h = cvx.matrix(np.vstack((np.zeros((N_train, 1)), C * np.ones((N_train, 1)))))
    A = cvx.matrix(1.0 * y_train[None, :])
    b = cvx.matrix(0.0)

    # use cvxopt library to solve QP problems
    result = cvx.solvers.qp(P, q, G, h, A, b)
    alpha = np.reshape(result["x"], N_train)
    alpha[alpha < C * epsilon] = 0
    alpha[alpha > C * (1 - epsilon)] = C

    # find bias parameter
    support_indices, = np.where(alpha != 0)
    active_indices, = np.where(np.logical_and(alpha != 0, alpha < C))
    w0 = np.mean(y_train[active_indices] * (
                1 - np.matmul(yyK[np.ix_(active_indices, support_indices)], alpha[support_indices])))

    globals()['f_pred%s' % i] = np.matmul(K_train, X_train[:, None] * alpha[:, None]) + w0

     pcost       dcost       gap    pres   dres
 0:  2.9600e+01 -4.2260e+04  8e+04  4e-01  3e-14
 1:  1.2992e+01 -7.4611e+03  8e+03  2e-02  4e-14
 2: -6.2643e+02 -2.9089e+03  2e+03  4e-03  4e-14
 3: -8.9105e+02 -1.7204e+03  8e+02  1e-03  4e-14
 4: -1.0255e+03 -1.3482e+03  3e+02  2e-04  4e-14
 5: -1.0893e+03 -1.1778e+03  9e+01  2e-05  4e-14
 6: -1.1096e+03 -1.1353e+03  3e+01  3e-07  5e-14
 7: -1.1169e+03 -1.1209e+03  4e+00  9e-14  5e-14
 8: -1.1184e+03 -1.1185e+03  1e-01  1e-14  5e-14
 9: -1.1185e+03 -1.1185e+03  4e-03  4e-14  5e-14
10: -1.1185e+03 -1.1185e+03  7e-05  5e-14  5e-14
Optimal solution found.


In [ ]:
#Prepare f prediction list and then create confusion matrix

f_lst = [f_pred1,f_pred2,f_pred3,f_pred4,f_pred5]
y_predicted = np.zeros(N_train).astype(int)

for i in range(N_train):
    max_f = max(f_lst[0][i],f_lst[1][i],f_lst[2][i],f_lst[3][i],f_lst[4][i])
    y_predicted[i] = np.where(max_f == f_lst)[0][0] + 1

y_train = labels[0:1000].astype(int)
confusion_matrix = pd.crosstab(np.reshape(y_predicted, N_train), y_train, rownames = ['y_predicted'], colnames = ['y_train'])
print(confusion_matrix)


In [ ]:
for i in range(1, 6):
    y_train = labels[0:1000].astype(int)
    y_train[y_train != i] = -1
    y_train[y_train == i] = 1

    s = 10
    K_train = gaussian_kernel(X_train, X_train, s)
    yyK = np.matmul(y_train[:, None], y_train[None, :]) * K_train

    K_test = gaussian_kernel(X_train, X_train, s)

    # set learning parameters
    C = 10
    epsilon = 1e-3

    P = cvx.matrix(yyK)
    q = cvx.matrix(-np.ones((N_train, 1)))
    G = cvx.matrix(np.vstack((-np.eye(N_train), np.eye(N_train))))
    h = cvx.matrix(np.vstack((np.zeros((N_train, 1)), C * np.ones((N_train, 1)))))
    A = cvx.matrix(1.0 * y_train[None, :])
    b = cvx.matrix(0.0)

    # use cvxopt library to solve QP problems
    result = cvx.solvers.qp(P, q, G, h, A, b)
    alpha = np.reshape(result["x"], N_train)
    alpha[alpha < C * epsilon] = 0
    alpha[alpha > C * (1 - epsilon)] = C

    # find bias parameter
    support_indices, = np.where(alpha != 0)
    active_indices, = np.where(np.logical_and(alpha != 0, alpha < C))
    w0 = np.mean(y_train[active_indices] * (
                1 - np.matmul(yyK[np.ix_(active_indices, support_indices)], alpha[support_indices])))

    globals()['f_test_pred%s' % i] = np.matmul(K_test, y_train[:, None] * alpha[:, None]) + w0

In [ ]:
#Prepare f test prediction list and then create confusion matrix

f_test_lst = [f_test_pred1,f_test_pred2,f_test_pred3,f_test_pred4,f_test_pred5]
y_test_predicted = np.zeros(N_test).astype(int)

for i in range(N_test):
    max_f = max(f_test_lst[0][i],f_test_lst[1][i],f_test_lst[2][i],f_test_lst[3][i],f_test_lst[4][i])
    y_test_predicted[i] = np.where(max_f == f_test_lst)[0][0] + 1

confusion_matrix = pd.crosstab(np.reshape(y_test_predicted, N_test), y_test, rownames = ['y_predicted'], colnames = ['y_train'])
print(confusion_matrix)